In [10]:
import torch
A_log = torch.arange(1, 7, dtype=torch.float32).repeat(3, 1)
A_log

tensor([[1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.]])

In [11]:
-torch.exp(A_log)

tensor([[  -2.7183,   -7.3891,  -20.0855,  -54.5981, -148.4132, -403.4288],
        [  -2.7183,   -7.3891,  -20.0855,  -54.5981, -148.4132, -403.4288],
        [  -2.7183,   -7.3891,  -20.0855,  -54.5981, -148.4132, -403.4288]])